In [97]:
%reload_ext lab_black
import pandas as pd
import numpy as np
from plotnine import *

In [98]:
# read data
od = pd.read_csv(
    "https://raw.githubusercontent.com/MIDS-at-Duke/estimating-impact-of-opioids-2020-purlple-team/coneel/10_code/od_deaths_state.csv?token=ARFW6V3OWJEBEXR6EAMOD4S7SS5MM"
)

population = pd.read_csv(
    "/Users/samsloate/Desktop/Data_Science/Opioids_Project/estimating-impact-of-opioids-2020-purlple-team/20_intermediate_files/countypopulations.csv"
)

In [99]:
# rename od columns to match
od = od.rename(columns={"County Code": "FIPS"})

# drop _merge column in popualtion dataset
population = population.drop(columns="_merge")

In [100]:
# merge dataset. Use inner merge, because it's okay if some years/counties don't have data in th od dataset. That just means there were less than 10 deaths
od_pop = pd.merge(population, od, on=["FIPS", "Year"], how="inner", indicator=True)

assert od_pop[od_pop["_merge"] == "both"].all

In [101]:
# convert years to integers
od_pop["Year"] = od_pop["Year"].astype("int")

In [102]:
od_pop.sample(25)

,Unnamed: 0,State Name,County_x,Year,Population,FIPS,State Abbr,County_y,State,Deaths,_merge
870,4435,Colorado,Boulder County,2015,318027,8013,CO,Boulder County,CO,32.0,both
5346,41292,Ohio,Lucas County,2012,436753,39095,OH,Lucas County,OH,90.0,both
4521,36633,New York,Nassau County,2013,1352193,36059,NY,Nassau County,NY,169.0,both
4517,36629,New York,Nassau County,2009,1332088,36059,NY,Nassau County,NY,118.0,both
3650,26852,Minnesota,Olmsted County,2012,146907,27109,MN,Olmsted County,MN,13.0,both
2915,22455,Louisiana,Orleans Parish,2015,389742,22071,LA,Orleans Parish,LA,96.0,both
3005,23067,Maine,Cumberland County,2007,278781,23005,ME,Cumberland County,ME,31.0,both
6822,51505,Texas,Fort Bend County,2005,462953,48157,TX,Fort Bend County,TX,13.0,both
7297,57367,Virginia,Pulaski County,2007,35058,51155,VA,Pulaski County,VA,12.0,both
2336,14333,Indiana,LaPorte County,2013,111423,18091,IN,La Porte County,IN,16.0,both


In [103]:
# drop unneeded columns

od_pop = od_pop.drop(columns=["County_y", "State", "_merge"])

In [104]:
# rename columns for easier understanding
od_pop = od_pop.rename(
    columns={"State Name": "State", "County_x": "County"},
    errors="raise",
)

# reorder columns
od_pop = od_pop[
    ["FIPS", "State Abbr", "State", "County", "Year", "Deaths", "Population"]
]

# sort by state, county
od_pop = od_pop.sort_values(["State", "County", "Year"])
od_pop.sample(25)

,FIPS,State Abbr,State,County,Year,Deaths,Population
7106,48485,TX,Texas,Wichita County,2009,26.0,130913
5186,39035,OH,Ohio,Cuyahoga County,2003,99.0,1359929
6391,45083,SC,South Carolina,Spartanburg County,2004,38.0,261828
6408,45091,SC,South Carolina,York County,2008,25.0,217463
2041,17093,IL,Illinois,Kendall County,2011,11.0,116814
3922,30029,MT,Montana,Flathead County,2008,11.0,90260
2430,18163,IN,Indiana,Vanderburgh County,2014,39.0,181934
4204,34023,NJ,New Jersey,Middlesex County,2014,114.0,824160
538,6053,CA,California,Monterey County,2013,50.0,426712
6419,46099,SD,South Dakota,Minnehaha County,2015,18.0,183439


In [105]:
# Create per capita deaths column
od_pop["Deaths Per 100,000 People"] = (od_pop["Deaths"] / od_pop["Population"]) * 100000

In [106]:
# create TX, FL, and WA datasets

# Florida: pre and post 2010
FL_policy_year = 2010
FL = od_pop[od_pop["State Abbr"] == "FL"]
FL_pre = FL[FL["Year"] < FL_policy_year]
FL_post = FL[FL["Year"] >= FL_policy_year]

# Texas: pre and post 2007
TX_policy_year = 2007
TX = od_pop[od_pop["State Abbr"] == "TX"]
TX_pre = TX[TX["Year"] < TX_policy_year]
TX_post = TX[TX["Year"] >= TX_policy_year]

# WA: pre and post 2012
WA_policy_year = 2012
WA = od_pop[od_pop["State Abbr"] == "FL"]
WA_pre = WA[WA["Year"] < WA_policy_year]
WA_post = WA[WA["Year"] >= WA_policy_year]

# create a list of the states
titles = od_pop["State"].unique()
titles = titles.tolist()

In [107]:
# graph pre-post: Florida
p = (
    ggplot()
    + geom_smooth(
        FL_pre,
        aes(x="Year", y="Deaths Per 100,000 People"),
        color="green",
        method="lm",
    )
    + geom_smooth(
        FL_post,
        aes(x="Year", y="Deaths Per 100,000 People"),
        color="green",
        method="lm",
    )
    + geom_vline(xintercept=2010, linetype="dotted")
    + xlab("Year")
    + ylab("Deaths Per 100,000 People")
    + theme_minimal()
    + scale_x_continuous(
        breaks=[2003, 2005, 2007, 2009, 2011, 2013, 2015], limits=[2003, 2015]
    )
    + labs(title="Overdose Deaths per Capita, Florida")
)
ggsave(
    plot=p,
    filename="FL_pre_post",
    path="/Users/samsloate/Desktop/Overdose_Deaths",
    dpi=100,
)

/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: /Users/samsloate/Desktop/Overdose_Deaths/FL_pre_post


In [108]:
# graph pre-post: Texas
p = (
    ggplot()
    + geom_smooth(
        TX_pre,
        aes(x="Year", y="Deaths Per 100,000 People"),
        color="green",
        method="lm",
    )
    + geom_smooth(
        TX_post,
        aes(x="Year", y="Deaths Per 100,000 People"),
        color="green",
        method="lm",
    )
    + geom_vline(xintercept=2007, linetype="dotted")
    + xlab("Year")
    + ylab("Deaths Per 100,000 People")
    + theme_minimal()
    + scale_x_continuous(
        breaks=[2003, 2005, 2007, 2009, 2011, 2013, 2015], limits=[2003, 2015]
    )
    + labs(title="Overdose Deaths per Capita, Texas")
)
ggsave(
    plot=p,
    filename="TX_pre_post",
    path="/Users/samsloate/Desktop/Overdose_Deaths",
    dpi=100,
)

/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: /Users/samsloate/Desktop/Overdose_Deaths/TX_pre_post


In [109]:
# graph pre-post: Washington
p = (
    ggplot()
    + geom_smooth(
        WA_pre,
        aes(x="Year", y="Deaths Per 100,000 People"),
        color="green",
        method="lm",
    )
    + geom_smooth(
        WA_post,
        aes(x="Year", y="Deaths Per 100,000 People"),
        color="green",
        method="lm",
    )
    + geom_vline(xintercept=2012, linetype="dotted")
    + xlab("Year")
    + ylab("Deaths Per 100,000 People")
    + theme_minimal()
    + scale_x_continuous(
        breaks=[2003, 2005, 2007, 2009, 2011, 2013, 2015], limits=[2003, 2015]
    )
    + labs(title="Overdose Deaths per Capita, Washington")
)
ggsave(
    plot=p,
    filename="WA_pre_post",
    path="/Users/samsloate/Desktop/Overdose_Deaths",
    dpi=100,
)

/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: /Users/samsloate/Desktop/Overdose_Deaths/WA_pre_post


In [110]:
# generate pre and post datasets in 2010, for Florida DiD
states = od_pop["State Abbr"].unique()
FL_df_pre = []
FL_df_post = []

# creates list of dataframes with pre 2010 data
for state in states:
    state_code = od_pop[od_pop["State Abbr"] == state]
    x = state_code[state_code["Year"] < FL_policy_year]
    # x will be a state for 2010 or lower years
    FL_df_pre.append(x)


# creates list of dataframes with post 2010 data
for state in states:
    state_code = od_pop[od_pop["State Abbr"] == state]
    x = state_code[state_code["Year"] >= FL_policy_year]
    # x will be a state for 2010 or lower years
    FL_df_post.append(x)

# run checks
assert len(FL_df_pre) == len(FL_df_post)
assert len(FL_df_pre) == 50

In [111]:
# generate pre and post datasets in 2007, for Texas DiD
states = od_pop["State Abbr"].unique()
TX_df_pre = []
TX_df_post = []

# creates list of dataframes with pre 2007 data
for state in states:
    state_code = od_pop[od_pop["State Abbr"] == state]
    x = state_code[state_code["Year"] < TX_policy_year]
    # x will be a state for 2010 or lower years
    TX_df_pre.append(x)


# creates list of dataframes with post 2007 data
for state in states:
    state_code = od_pop[od_pop["State Abbr"] == state]
    x = state_code[state_code["Year"] >= TX_policy_year]
    # x will be a state for 2010 or lower years
    TX_df_post.append(x)

# run checks
assert len(TX_df_pre) == len(TX_df_post)
assert len(TX_df_pre) == 50

In [112]:
# generate pre and post datasets in 2012, for WA DiD
states = od_pop["State Abbr"].unique()
WA_df_pre = []
WA_df_post = []

# creates list of dataframes with pre 2012 data
for state in states:
    state_code = od_pop[od_pop["State Abbr"] == state]
    x = state_code[state_code["Year"] < WA_policy_year]
    # x will be a state for 2010 or lower years
    WA_df_pre.append(x)


# creates list of dataframes with post 2012 data
for state in states:
    state_code = od_pop[od_pop["State Abbr"] == state]
    x = state_code[state_code["Year"] >= WA_policy_year]
    # x will be a state for 2010 or lower years
    WA_df_post.append(x)

# run checks
assert len(WA_df_pre) == len(WA_df_post)
assert len(WA_df_pre) == 50

In [121]:
# run a loop to generate DiD comparisons to Florida
pltList = []

for number in range(0, 50):
    p = (
        ggplot()
        + geom_smooth(
            FL_df_pre[number],
            aes(x="Year", y="Deaths Per 100,000 People"),
            color="skyblue",
            method="lm",
        )
        + geom_smooth(
            FL_df_post[number],
            aes(x="Year", y="Deaths Per 100,000 People"),
            color="skyblue",
            method="lm",
        )
        + geom_smooth(
            FL_pre,
            aes(x="Year", y="Deaths Per 100,000 People"),
            color="green",
            method="lm",
        )
        + geom_smooth(
            FL_post,
            aes(x="Year", y="Deaths Per 100,000 People"),
            color="green",
            method="lm",
        )
        + geom_vline(xintercept=FL_policy_year, linetype="dotted")
        + xlab("Year")
        + ylab("Deaths Per 100,000 People")
        + theme_minimal()
        + scale_x_continuous(
            breaks=[2003, 2005, 2007, 2009, 2011, 2013, 2015], limits=[2003, 2015]
        )
        + labs(title=titles[number] + " vs. Florida, Overdose Deaths per Capita")
    )
    pltList.append(p)
    ggsave(
        plot=p,
        filename=titles[number],
        path="/Users/samsloate/Desktop/Overdose_Deaths",
        dpi=100,
    )

/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: /Users/samsloate/Desktop/Overdose_Deaths/Alabama
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: /Users/samsloate/Desktop/Overdose_Deaths/Arizona
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: /Users/samsloate/Desktop/Overdose_Deaths/Arkansas
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/samsloa

In [124]:
# run a loop to generate DiD comparisons to TX
pltList = []

for number in range(0, 50):
    p = (
        ggplot()
        + geom_smooth(
            TX_df_pre[number],
            aes(x="Year", y="Deaths Per 100,000 People"),
            color="skyblue",
            method="lm",
        )
        + geom_smooth(
            TX_df_post[number],
            aes(x="Year", y="Deaths Per 100,000 People"),
            color="skyblue",
            method="lm",
        )
        + geom_smooth(
            TX_pre,
            aes(x="Year", y="Deaths Per 100,000 People"),
            color="green",
            method="lm",
        )
        + geom_smooth(
            TX_post,
            aes(x="Year", y="Deaths Per 100,000 People"),
            color="green",
            method="lm",
        )
        + geom_vline(xintercept=TX_policy_year, linetype="dotted")
        + xlab("Year")
        + ylab("Deaths Per 100,000 People")
        + theme_minimal()
        + scale_x_continuous(
            breaks=[2003, 2005, 2007, 2009, 2011, 2013, 2015], limits=[2003, 2015]
        )
        + labs(title=titles[number] + " vs. Texas, Overdose Deaths per Capita")
    )
    pltList.append(p)
    ggsave(
        plot=p,
        filename=titles[number],
        path="/Users/samsloate/Desktop/Overdose_Deaths/Texas",
        dpi=100,
    )

/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: /Users/samsloate/Desktop/Overdose_Deaths/Texas/Alabama
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: /Users/samsloate/Desktop/Overdose_Deaths/Texas/Arizona
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: /Users/samsloate/Desktop/Overdose_Deaths/Texas/Arkansas
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in ima

In [125]:
# run a loop to generate DiD comparisons to TX
pltList = []

for number in range(0, 50):
    p = (
        ggplot()
        + geom_smooth(
            WA_df_pre[number],
            aes(x="Year", y="Deaths Per 100,000 People"),
            color="skyblue",
            method="lm",
        )
        + geom_smooth(
            WA_df_post[number],
            aes(x="Year", y="Deaths Per 100,000 People"),
            color="skyblue",
            method="lm",
        )
        + geom_smooth(
            WA_pre,
            aes(x="Year", y="Deaths Per 100,000 People"),
            color="green",
            method="lm",
        )
        + geom_smooth(
            WA_post,
            aes(x="Year", y="Deaths Per 100,000 People"),
            color="green",
            method="lm",
        )
        + geom_vline(xintercept=WA_policy_year, linetype="dotted")
        + xlab("Year")
        + ylab("Deaths Per 100,000 People")
        + theme_minimal()
        + scale_x_continuous(
            breaks=[2003, 2005, 2007, 2009, 2011, 2013, 2015], limits=[2003, 2015]
        )
        + labs(title=titles[number] + " vs. Washington, Overdose Deaths per Capita")
    )
    pltList.append(p)
    ggsave(
        plot=p,
        filename=titles[number],
        path="/Users/samsloate/Desktop/Overdose_Deaths/Washington",
        dpi=100,
    )

/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: /Users/samsloate/Desktop/Overdose_Deaths/Washington/Alabama
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: /Users/samsloate/Desktop/Overdose_Deaths/Washington/Arizona
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: /Users/samsloate/Desktop/Overdose_Deaths/Washington/Arkansas
/Users/samsloate/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6